# Hyperparameters search

In past notebooks, we pointing out that some models' parameters had an impact on the statistical performance of the models. Usually, we would like to optimize these parameters such that the model trained is as good as possible. This optimization is called hyperparameters tuning.

In this notebook, we will show a couple of method allowing to tune models' hyperparameters.

## Introductory example

We will take an example that we showed in the linear model where we discussed the impact of the $\alpha$ parameter on a `Ridge` model. Indeed, we mentioned that this parameter allows to regularize more or less the model. However, there is no general rule specifying what is a good $\alpha$ value. Indeed, it would depend of the dataset.

Let's load a dataset to tackle a regression problem.

In [1]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [2]:
y.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64

Now, we will define a `Ridge` model where we will process the the data with add some interaction between features using a `PolynomialFeatures` transformer.

In [3]:
import sklearn

sklearn.set_config(display="diagram")

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

model = make_pipeline(
    PolynomialFeatures(),
    StandardScaler(),
    Ridge(),
)
model

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()), ('ridge', Ridge())])

However, we did not change any of the default parameters given by scikit-learn. Let's evaluate this vanilla model.

In [5]:
import pandas as pd
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, X, y)
cv_results = pd.DataFrame(cv_results)

In [6]:
cv_results

,fit_time,score_time,test_score
0,0.132578,0.018565,0.467650
1,0.099072,0.007991,0.552113
2,0.086675,0.014852,0.579568
3,0.095193,0.009836,0.500778
4,0.052034,0.006014,-4.211175


In [7]:
cv_results.aggregate(["mean", "std"])

,fit_time,score_time,test_score
mean,0.093110,0.011451,-0.422213
std,0.028837,0.005156,2.118542


At this stage, there is nothing to tell use that our pipeline is the best pipeline that we could get. Indeed, we could imagine that the degree of the `PolynomialFeatures` could be higher or that the `Ridge` regressor should be more regularized. Let's check which parameters we could tune with the model:

In [8]:
for params in model.get_params():
    print(params)

memory
steps
verbose
polynomialfeatures
standardscaler
ridge
polynomialfeatures__degree
polynomialfeatures__include_bias
polynomialfeatures__interaction_only
polynomialfeatures__order
standardscaler__copy
standardscaler__with_mean
standardscaler__with_std
ridge__alpha
ridge__copy_X
ridge__fit_intercept
ridge__max_iter
ridge__normalize
ridge__positive
ridge__random_state
ridge__solver
ridge__tol


Two important parameters of this model are `polynomialfeatures__degree` and `ridge__alpha`. We will try to find the optimal values of these parameters for the current dataset.

## Manual hyperparameters search

Before to show the automated tools allowing to make hyperparameters tuning in scikit-learn, we will manually make our own manual simplified version.

<div class="alert alert-success">
    <b>EXERCISE</b>:
    <ul>
        <li>Split the dataset into a training and testing set.</li>
        <li>Make a nested <tt>for</tt> loop to try all the possible parameters combination that we defined in <tt>parameter_grid</tt>.</li>
        <li>In the internal loop, use a cross-validation (using <tt>cross_val_score</tt>) on the training set to get a distribution of score.</li>
        <li>Compute the mean and standard deviation of the cross-validation score and pick-up the best set of hyperparameters.</li>
        <li>Retrain a model with the combination of the best hyperparameters and evalute it on the testing set.</li>
    </ul>
</div>

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0
)

In [14]:
# %load solutions/solution_01.py
import numpy as np

parameter_grid = {
    "polynomialfeatures__degree": np.arange(2, 5),
    "ridge__alpha": np.logspace(1, 3, num=5),
}


In [20]:
models_results = []
best_score = 0.0
best_params = {}
for polynomialfeatures__degree in parameter_grid["polynomialfeatures__degree"]:
    for ridge__alpha in parameter_grid["ridge__alpha"]:
        model.set_params(
            polynomialfeatures__degree = polynomialfeatures__degree,
            ridge__alpha = ridge__alpha
        )
        cv_results = cross_validate(model, X_train, y_train)
        cv_results = pd.DataFrame(cv_results)
        cv_results = cv_results.aggregate(["mean", "std"])
        score = cv_results["test_score"][0]
        if score > best_score:
            best_score = score
            best_params = {
                "polynomialfeatures__degree": polynomialfeatures__degree,
                "ridge__alpha": ridge__alpha
            }
print(best_score)
print(best_params)

0.615069268592654
{'polynomialfeatures__degree': 2, 'ridge__alpha': 316.22776601683796}


In [21]:
# %load solutions/solution_02.py

In [22]:
# %load solutions/solution_03.py

In [23]:
# %load solutions/solution_04.py

In [24]:
# %load solutions/solution_05.py

In [25]:
# %load solutions/solution_06.py

In [26]:
# %load solutions/solution_07.py

## Hyperparameters search using a grid

The search that we performed manually is indeed known as a grid-search: we try every possible combination of the parameter that we first provided. Scikit-learn provides a specific estimator that will make the processing that we did previously: during `fit`, it will perform a cross-validation and pick the optimal hyperparameters using cross-validation.

In [27]:
from sklearn.model_selection import GridSearchCV

search_cv = GridSearchCV(model, param_grid=parameter_grid)
search_cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures(degree=4)),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge(alpha=1000.0))]),
             param_grid={'polynomialfeatures__degree': array([2, 3, 4]),
                         'ridge__alpha': array([  10.        ,   31.6227766 ,  100.        ,  316.22776602,
       1000.        ])})

We can get the best found parameters by looking at the fitted attributes `best_params_`:

In [28]:
search_cv.best_params_

{'polynomialfeatures__degree': 2, 'ridge__alpha': 316.22776601683796}

We can even get more information regarding the different combinations of hyperparameters tried during `fit` by looking at the fitted attribute `cv_results_`:

In [29]:
cv_results = pd.DataFrame(search_cv.cv_results_)
cv_results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_polynomialfeatures__degree,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.059990,0.007550,0.010683,0.001497,2,10.0,"{'polynomialfeatures__degree': 2, 'ridge__alph...",0.655806,0.528570,0.362651,0.653063,0.665514,0.573121,0.116659,7
1,0.088141,0.021528,0.015566,0.006488,2,31.622777,"{'polynomialfeatures__degree': 2, 'ridge__alph...",0.657668,0.565873,0.422209,0.654168,0.662040,0.592392,0.092297,5
2,0.081634,0.027884,0.009451,0.002830,2,100.0,"{'polynomialfeatures__degree': 2, 'ridge__alph...",0.639703,0.604569,0.494025,0.665001,0.655253,0.611710,0.062322,3
3,0.062829,0.008676,0.009173,0.001300,2,316.227766,"{'polynomialfeatures__degree': 2, 'ridge__alph...",0.601179,0.629179,0.562315,0.640136,0.642539,0.615069,0.030193,1
4,0.064712,0.016330,0.010846,0.003789,2,1000.0,"{'polynomialfeatures__degree': 2, 'ridge__alph...",0.576208,0.615647,0.592287,0.585586,0.619774,0.597900,0.017012,4


In addition, at the end of the `fit` procedure, if the parameter `refit` is set to `True` (default), a model with the best combination will be trained (as we did in the manual hyperparameters search). We can check this model by looking at the fitted attribute `best_estimator_`

In [30]:
search_cv.best_estimator_

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('ridge', Ridge(alpha=316.22776601683796))])

Indeed, this `best_estimator_` will be used when calling `predict` and `score` from the `GridSearchCV` instance.

In [31]:
search_cv.score(X_test, y_test)

-1.2224173460530392

<div class="alert alert-success">
    <b>EXERCISE</b>:
    <br>
    Since a <tt>GridSearchCV</tt> behave like any classifier or regressor, it can be used and evaluated by cross-validation. Use <tt>cross_validate</tt> to evaluate the previous grid-search model that we created.
</div>

In [35]:
# %load solutions/solution_08.py
cv_results = cross_validate(search_cv, X, y, return_estimator=True)
cv_results = pd.DataFrame(cv_results)
cv_results


,fit_time,score_time,estimator,test_score
0,22.879226,0.016091,GridSearchCV(estimator=Pipeline(steps=[('polyn...,0.258175
1,27.003312,0.011391,GridSearchCV(estimator=Pipeline(steps=[('polyn...,0.475082
2,28.198821,0.012727,GridSearchCV(estimator=Pipeline(steps=[('polyn...,0.561609
3,32.067886,0.018464,GridSearchCV(estimator=Pipeline(steps=[('polyn...,0.526413
4,30.300900,0.060819,GridSearchCV(estimator=Pipeline(steps=[('polyn...,-17.880146


In [36]:
# %load solutions/solution_09.py
cv_results["estimator"]


0    GridSearchCV(estimator=Pipeline(steps=[('polyn...
1    GridSearchCV(estimator=Pipeline(steps=[('polyn...
2    GridSearchCV(estimator=Pipeline(steps=[('polyn...
3    GridSearchCV(estimator=Pipeline(steps=[('polyn...
4    GridSearchCV(estimator=Pipeline(steps=[('polyn...
Name: estimator, dtype: object

In [37]:
# %load solutions/solution_10.py
for est in cv_results["estimator"]:
    print(est.best_params_)


{'polynomialfeatures__degree': 2, 'ridge__alpha': 31.622776601683793}
{'polynomialfeatures__degree': 2, 'ridge__alpha': 1000.0}
{'polynomialfeatures__degree': 2, 'ridge__alpha': 1000.0}
{'polynomialfeatures__degree': 2, 'ridge__alpha': 1000.0}
{'polynomialfeatures__degree': 3, 'ridge__alpha': 100.0}


<div class="alert alert-success">
    <b>QUESTION</b>:
    <br>
    Which limitation the grid-search approach suffer from?
</div>

## Randomized hyperparameters search

In the previous strategy, the grid-search has two limitations:

- it only explores combination of parameters defined in the grid;
- adding new parameters and values to explore will increase rapidly the cost of the search.

`RandomizedSearchCV` allows to specify a distribution from which to draw parameter values. It allows to explore the hyperparamters space on a non-grid fashion and as a user, you can give a butget of the number of combination you want to try.

In [38]:
from scipy.stats import loguniform

parameter_distributions = {
    "polynomialfeatures__degree": np.arange(1, 5),
    "ridge__alpha": loguniform(1, 3),
}

In [39]:
from sklearn.model_selection import RandomizedSearchCV

search_cv = RandomizedSearchCV(
    model, param_distributions=parameter_distributions, n_iter=10,
)

In [40]:
cv_results = cross_validate(search_cv, X, y, return_estimator=True)
cv_results = pd.DataFrame(cv_results)

In [41]:
cv_results

,fit_time,score_time,estimator,test_score
0,17.436294,0.008023,RandomizedSearchCV(estimator=Pipeline(steps=[(...,0.548827
1,14.613464,0.006106,RandomizedSearchCV(estimator=Pipeline(steps=[(...,0.468163
2,12.561264,0.008435,RandomizedSearchCV(estimator=Pipeline(steps=[(...,0.550824
3,8.406125,0.006109,RandomizedSearchCV(estimator=Pipeline(steps=[(...,0.536888
4,7.866972,0.013089,RandomizedSearchCV(estimator=Pipeline(steps=[(...,0.660527


In [42]:
for est in cv_results["estimator"]:
    print(est.best_params_)

{'polynomialfeatures__degree': 1, 'ridge__alpha': 1.9065236364446558}
{'polynomialfeatures__degree': 1, 'ridge__alpha': 1.7531638814255666}
{'polynomialfeatures__degree': 1, 'ridge__alpha': 2.1998547700424167}
{'polynomialfeatures__degree': 1, 'ridge__alpha': 1.4416660694131567}
{'polynomialfeatures__degree': 1, 'ridge__alpha': 2.6301686159766344}


## Model with internal hyperparameter tuning

Some classifiers or regressors come with the some efficient hyperparameter selection, at least more efficient than a grid-search. Usually, the name of the classsifiers or regressors finish with `CV` (e.g. `RidgeCV`).

<div class="alert alert-success">
    <b>EXERCISE</b>:
    <br>
    <ul>
        <li>Create a pipeline made of a <tt>PolynomialFeatures</tt>, a <tt>StandardScaler</tt>, and a <tt>Ridge</tt>.</li>
        <li>Create a grid-search by passing the previous pipeline and tune the parameter <tt>alpha</tt> such that you will try the values <tt>np.logspace(-2, 2, num=50)</tt>.</li>
        <li>Fit the grid-search on the training set and check the time it takes.</li>
        <li>Repeat the experiment by replacing the <tt>Ridge</tt> regressor by a <tt>RidgeCV</tt> regressor and removing the <tt>GridSearchCV</tt>.</li>
    </ul>
    Which approach is more efficient in terms of computational performance.
</div>

In [50]:
# %load solutions/solution_11.py
from sklearn.linear_model import Ridge

alphas = np.logspace(-2, 2, num=50)

model = GridSearchCV(
    make_pipeline(
        PolynomialFeatures(),
        StandardScaler(),
        Ridge(),
    ),
    param_grid={
        "ridge__alpha": alphas
    },
    scoring="neg_mean_squared_error",
)
model


GridSearchCV(estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-02, 1.20679264e-02, 1.45634848e-02, 1.75751062e-02,
       2.12095089e-02, 2.55954792e-02, 3.08884360e-02, 3.72759372e-02,
       4.49843267e-02, 5.42867544e-02, 6.55128557e-02, 7.90604321e-...
       1.93069773e+00, 2.32995181e+00, 2.81176870e+00, 3.39322177e+00,
       4.09491506e+00, 4.94171336e+00, 5.96362332e+00, 7.19685673e+00,
       8.68511374e+00, 1.04811313e+01, 1.26485522e+01, 1.52641797e+01,
       1.84206997e+01, 2.22299648e+01, 2.68269580e+01, 3.23745754e+01,
       3.90693994e+01, 4.71486636e+01, 5.68986603e+01, 6.86648845e+01,
       8.28642773e+01, 1.00000000e+02])},
             scoring='neg_mean_squared_error')

In [53]:
%%time
# %load solutions/solution_12.py

model.fit(X_train, y_train)


Wall time: 26.6 s


GridSearchCV(estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-02, 1.20679264e-02, 1.45634848e-02, 1.75751062e-02,
       2.12095089e-02, 2.55954792e-02, 3.08884360e-02, 3.72759372e-02,
       4.49843267e-02, 5.42867544e-02, 6.55128557e-02, 7.90604321e-...
       1.93069773e+00, 2.32995181e+00, 2.81176870e+00, 3.39322177e+00,
       4.09491506e+00, 4.94171336e+00, 5.96362332e+00, 7.19685673e+00,
       8.68511374e+00, 1.04811313e+01, 1.26485522e+01, 1.52641797e+01,
       1.84206997e+01, 2.22299648e+01, 2.68269580e+01, 3.23745754e+01,
       3.90693994e+01, 4.71486636e+01, 5.68986603e+01, 6.86648845e+01,
       8.28642773e+01, 1.00000000e+02])},
             scoring='neg_mean_squared_error')

In [54]:
# %load solutions/solution_13.py
from sklearn.linear_model import RidgeCV

model = make_pipeline(
    PolynomialFeatures(),
    StandardScaler(),
    RidgeCV(alphas=alphas, store_cv_values=True),
)


In [55]:
%%time
# %load solutions/solution_14.py

model.fit(X_train, y_train)


Wall time: 1.58 s


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('ridgecv',
                 RidgeCV(alphas=array([1.00000000e-02, 1.20679264e-02, 1.45634848e-02, 1.75751062e-02,
       2.12095089e-02, 2.55954792e-02, 3.08884360e-02, 3.72759372e-02,
       4.49843267e-02, 5.42867544e-02, 6.55128557e-02, 7.90604321e-02,
       9.54095476e-02, 1.15139540e-01, 1.38949549e...
       1.93069773e+00, 2.32995181e+00, 2.81176870e+00, 3.39322177e+00,
       4.09491506e+00, 4.94171336e+00, 5.96362332e+00, 7.19685673e+00,
       8.68511374e+00, 1.04811313e+01, 1.26485522e+01, 1.52641797e+01,
       1.84206997e+01, 2.22299648e+01, 2.68269580e+01, 3.23745754e+01,
       3.90693994e+01, 4.71486636e+01, 5.68986603e+01, 6.86648845e+01,
       8.28642773e+01, 1.00000000e+02]),
                         store_cv_values=True))])

## Inspection of hyperparameters in cross-validation

Sometimes, we perform a search cross-validation inside a cross-validation evaluation. In this case, we potentially have different set of hyperparameter values for each individual cross-validation split. We can indeed inspect these values. Let's define a `GridSearchCV` model.

In [43]:
from sklearn.linear_model import Ridge

alphas = np.logspace(-2, 2, num=50)

model = GridSearchCV(
    make_pipeline(
        PolynomialFeatures(),
        StandardScaler(),
        Ridge(),
    ),
    param_grid={
        "ridge__alpha": alphas
    },
    scoring="neg_mean_squared_error",
)
model

GridSearchCV(estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('standardscaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([1.00000000e-02, 1.20679264e-02, 1.45634848e-02, 1.75751062e-02,
       2.12095089e-02, 2.55954792e-02, 3.08884360e-02, 3.72759372e-02,
       4.49843267e-02, 5.42867544e-02, 6.55128557e-02, 7.90604321e-...
       1.93069773e+00, 2.32995181e+00, 2.81176870e+00, 3.39322177e+00,
       4.09491506e+00, 4.94171336e+00, 5.96362332e+00, 7.19685673e+00,
       8.68511374e+00, 1.04811313e+01, 1.26485522e+01, 1.52641797e+01,
       1.84206997e+01, 2.22299648e+01, 2.68269580e+01, 3.23745754e+01,
       3.90693994e+01, 4.71486636e+01, 5.68986603e+01, 6.86648845e+01,
       8.28642773e+01, 1.00000000e+02])},
             scoring='neg_mean_squared_error')

Then, we can run a cross-validation by passing the model to `cross_validate`. In addition, we can store every model train on each cross-validation splits by setting `return_estimator` to `True`.

In [44]:
cv_results = cross_validate(model, X, y, cv=3, return_estimator=True)
cv_results = pd.DataFrame(cv_results)
cv_results

,fit_time,score_time,estimator,test_score
0,21.587997,0.019964,GridSearchCV(estimator=Pipeline(steps=[('polyn...,-0.693117
1,21.541095,0.013996,GridSearchCV(estimator=Pipeline(steps=[('polyn...,-0.493655
2,22.249150,0.021013,GridSearchCV(estimator=Pipeline(steps=[('polyn...,-9.096473


We see that the `estimator` columns contain the different estimators. Thus we can check the `best_params_` stored by the `GridSearchCV`.

In [45]:
for estimator_cv_fold in cv_results["estimator"]:
    print(estimator_cv_fold.best_params_)

{'ridge__alpha': 22.229964825261934}
{'ridge__alpha': 0.20235896477251566}
{'ridge__alpha': 1.0985411419875584}


Such inspection allows to study the stability of the hyperparameter values.